In [1]:
# Алексеев Д.П. (DSU-4,FEML-8)
# Домашнее задание к лекции «Функции потерь и оптимизация» (#3). Скорректированное

# Задание:
# Прочитать про методы оптимизации для нейронных сетей https://habr.com/post/318970/
# Реализовать самостоятельно логистическую регрессию

# Обучить ее методом градиентного спуска:
# -Методом nesterov momentum
# -Методом rmsprop

In [2]:
import pandas as pd
import numpy as np

In [3]:
# Зададим произвольную линейную функцию, коэффициенты которой будем подбирать методом градиентного спуска,
# используя для этого функцию логистической регрессии
N = 100
np.random.seed(21)
x1 = np.random.uniform(low=0, high=5, size=N)
x2 = np.random.uniform(low=0, high=5, size=N)
y = 2*x1 + 3*x2 + 1
# соответственно, при подборе (в идеале) должны получиться следующие коэффициенты:
# params[0] =1 
# params[1] при х1 =2
# params[2] при х2 =3 

x1[:5], x2[:5], y[:5]

(array([0.2436244 , 1.4455483 , 3.60483173, 0.10808125, 1.02961383]),
 array([3.45260891, 3.89623315, 0.23614565, 0.37280601, 3.02422269]),
 array([11.84507555, 15.57979604,  8.91810042,  2.33458052, 12.13189574]))

In [4]:
# определим функцию логистической кривой (сигмоиды)
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [5]:
EPOCHS = 200
lr = 0.0001
loss_functions = []
params = []
probabilities = []
np.random.seed(21)
params = np.random.normal(size=(3,)) #зададим начальную точку параметров оптимизации (градиентного спуска)
params

array([-0.05196425, -0.11119605,  1.0417968 ])

In [6]:
for _ in range(EPOCHS):
    # рассчитаем прогнозную вероятность правильного значения, 
    # т.е. совпадающего со значением 'y', рассчитанным по формуле y = 2*x1 + 3*x2 + 1
    predict_probability = sigmoid(params[0] + params[1] * x1 + params[2] * x2)
    probabilities.append(predict_probability)
    
    # зададим функцию потерь (отклонений прогнозируемых значений от истинных значений 'y')
    loss_function = - np.sum(y * np.log(predict_probability) + (1 - y) * np.log(1 - predict_probability)) / N
    loss_functions.append(loss_function)
    
    # уменьшим значения параметров на величину градиента
    params[0] -= lr * np.sum((predict_probability - y) / len(probabilities))
    params[1] -= lr * np.sum((predict_probability - y) * x1 / len(probabilities))
    params[2] -= lr * np.sum((predict_probability - y) * x2 / len(probabilities))

In [7]:
params
# Вывод: Как видно из результатов градиентного спуска, параметры подобраны не совсем оптимально 
# (к тому же выдается ошибка деления на ноль, хотя непонятно, 
# откуда в знаменателе мог взяться ноль - в формуле идёт деление на величину выборки N=100).

# Но "тренд" в целом определен верно:
# y = 2.01644721*x1 + 3.24780056*x2 + 0.69582679

# при этом целевая функция была: y = 2*x1 + 3*x2 + 1

array([0.69582679, 2.01644721, 3.24780056])

In [8]:
# из описания на Хабре (https://habr.com/post/318970/),
# если я правильно понял, метод Nesterov Momentum -  по сути, то же самое, что обычный градиентный спуск, 
# но дополнительно нужно запоминать накопленный момент:
# «Если мы некоторое время движемся в определённом направлении, то, вероятно, 
# нам следует туда двигаться некоторое время и в будущем». 
# Для этого нужно уметь обращаться к недавней истории изменений каждого параметра.

# Т.е. нам нужно запоминать предыдущее направление(значение параметров) и умножать на некий коэффициент "сохранения момента",
# чтобы оптимизироваться быстрее. (В статье на Хабре указано, что "обычно берётся порядка 0.9")
momentum = 0.9 

# задаем переменные для сохранения предыдущих значений свободного члена (t0) и параметров при x1,х2:
t0 = 0
t1 = 0
t2 = 0

# уменьшим кол-во эпох вдвое (до 100), остальные параметры оптимизации оставим прежними 
EPOCHS = 100
lr = 0.0001
loss_functions_n = []
params_n = []
probabilities_n = []
np.random.seed(21)
params_n = np.random.normal(size=(3,)) #зададим начальную точку параметров оптимизации (градиентного спуска)
params_n

array([-0.05196425, -0.11119605,  1.0417968 ])

In [9]:
for _ in range(EPOCHS):
    previous_t0 = t0
    previous_t1 = t1
    previous_t2 = t2
    
    # рассчитаем прогнозную вероятность правильного значения, 
    # т.е. совпадающего со значением 'y', рассчитанным по формуле y = 2*x1 + 3*x2 + 1
    predict_probability = sigmoid(params_n[0] + params_n[1] * x1 + params_n[2] * x2)
    probabilities_n.append(predict_probability)
    
    # зададим функцию потерь (отклонений прогнозируемых значений от истинных значений 'y')
    loss_function = - np.sum(y * np.log(predict_probability) + (1 - y) * np.log(1 - predict_probability)) / N
    loss_functions_n.append(loss_function)
    
    # вместо обычного вычитания градиента вычтем градиент+накопленный момент 
    #(коэфф-т сохранения момента, умноженный на предыдущие значения параметров)    
    params_n[0] += -(momentum * previous_t0 + lr * np.sum((predict_probability - y) / len(probabilities_n)))
    params_n[1] += -(momentum * previous_t1 + lr * np.sum((predict_probability - y) * x1 / len(probabilities_n)))
    params_n[2] += -(momentum * previous_t2 + lr * np.sum((predict_probability - y) * x2 / len(probabilities_n)))

In [10]:
params_n
# Вывод: при вдвое меньшем кол-ве эпох коэфф-ты определены почти так же точно, как при обычном градиентном спуске:
# y = 1.76715813*x1 + 2.98888303*x2 + 0.60822561

# обычный градиентный спуск(100 эпох): y = 2.01644721*x1 + 3.24780056*x2 + 0.69582679

# при этом целевая функция была: y = 2*x1 + 3*x2 + 1

# Обращает на себя внимание, что при одинаковом кол-ве эпох = 100 оба метода всё же выдают абсолютно одинаковые коэфф-ты. 
# Возможно, что отличаются затраты времени на оптимизацию (не замерял).
# Видимо, эффективность метода Nesterov Momentum будет более заметна на сложных функциях.

array([0.60822561, 1.76715813, 2.98888303])

In [ ]:
# из описания на Хабре (https://habr.com/post/318970/)
# метод RMSProp
# def RMSProp(X, gamma, lr=0.25, eps=0.00001):
#     Y = []
#     EG = 0
#     for x in X:
#         EG = gamma*EG + (1-gamma)*x*x
#         v = lr/np.sqrt(EG + eps)*x
#         Y.append(v)
#     return np.asarray(Y)

In [11]:
# gamma - это коэфф-т сохранения момента ('momentum' в методе Nesterov Momentum) 
gamma = 0.9 

# eps - это эпсилон, введённый для исключения ситуации деления на ноль
eps=0.00001

# задаем переменные для сохранения предыдущих значений свободного члена (t0) и параметров при x1,х2:
t0 = 0
t1 = 0
t2 = 0

# остальные параметры оптимизации оставим прежними 
EPOCHS = 100
lr = 0.0001
loss_functions_rms = []
params_rms = []
probabilities_rms = []
np.random.seed(21)
params_rms = np.random.normal(size=(3,)) #зададим начальную точку параметров оптимизации (градиентного спуска)
params_rms

array([-0.05196425, -0.11119605,  1.0417968 ])